In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# 한글 폰트 설정 (Windows)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

class SeoulBikeAnalyzer:
    def __init__(self):
        self.data = None
        self.station_data = None
        
    def generate_sample_data(self):
        """실제 따릉이 데이터가 없을 때 샘플 데이터 생성"""
        np.random.seed(42)
        
        # 날짜 범위 생성 (최근 6개월)
        start_date = datetime(2024, 1, 1)
        end_date = datetime(2024, 6, 30)
        dates = pd.date_range(start_date, end_date, freq='h')
        
        # 대여소 정보
        stations = [
            '여의도공원', '한강공원', '홍대입구역', '강남역', '잠실역',
            '종로3가역', '명동역', '이태원역', '성수역', '건대입구역'
        ]
        
        # 데이터 생성
        data = []
        for date in dates[:1000]:  # 1000개 샘플
            for station in np.random.choice(stations, size=3):
                # 시간대별 패턴 반영
                hour = date.hour
                base_rental = 10
                
                # 출퇴근 시간대 패턴
                if hour in [7, 8, 9, 18, 19, 20]:
                    base_rental *= 3
                elif hour in [12, 13, 14]:  # 점심시간
                    base_rental *= 2
                elif hour in [22, 23, 0, 1, 2, 3, 4, 5]:  # 심야시간
                    base_rental *= 0.3
                
                # 요일별 패턴
                if date.weekday() < 5:  # 평일
                    base_rental *= 1.5
                else:  # 주말
                    base_rental *= 1.2
                
                # 계절별 패턴
                if date.month in [3, 4, 5, 9, 10]:  # 봄, 가을
                    base_rental *= 1.3
                elif date.month in [6, 7, 8]:  # 여름
                    base_rental *= 0.8
                else:  # 겨울
                    base_rental *= 0.6
                
                rental_count = max(0, int(np.random.poisson(base_rental)))
                return_count = max(0, int(np.random.poisson(base_rental * 0.9)))
                
                data.append({
                    'datetime': date,
                    'station_name': station,
                    'rental_count': rental_count,
                    'return_count': return_count,
                    'hour': hour,
                    'day_of_week': date.weekday(),
                    'month': date.month,
                    'is_weekend': date.weekday() >= 5
                })
        
        self.data = pd.DataFrame(data)
        return self.data
    
    def basic_analysis(self):
        """기본 분석 수행"""
        if self.data is None:
            print("데이터가 없습니다. generate_sample_data()를 먼저 실행하세요.")
            return
        
        print("🚲 서울시 따릉이 데이터 분석")
        print("=" * 60)
        
        # 기본 정보
        print(f"📊 데이터 기본 정보:")
        print(f"   • 총 레코드 수: {len(self.data):,}개")
        print(f"   • 분석 기간: {self.data['datetime'].min().date()} ~ {self.data['datetime'].max().date()}")
        print(f"   • 대여소 수: {self.data['station_name'].nunique()}개")
        print(f"   • 총 대여 횟수: {self.data['rental_count'].sum():,}회")
        print(f"   • 총 반납 횟수: {self.data['return_count'].sum():,}회")
        print()
        
        # 기본 통계
        print("📈 기본 통계:")
        stats = self.data[['rental_count', 'return_count']].describe()
        print(stats.round(2))
        print()
        
        return stats
    
    def time_analysis(self):
        """시간대별 분석"""
        # 시간대별 평균 대여량
        hourly_avg = self.data.groupby('hour')[['rental_count', 'return_count']].mean()
        
        print("⏰ 시간대별 평균 대여/반납량:")
        print("시간  대여   반납")
        print("-" * 20)
        for hour in range(24):
            rental = hourly_avg.loc[hour, 'rental_count']
            return_val = hourly_avg.loc[hour, 'return_count']
            print(f"{hour:2d}시  {rental:5.1f}  {return_val:5.1f}")
        print()
        
        # 피크 시간대 찾기
        peak_hour = hourly_avg['rental_count'].idxmax()
        peak_value = hourly_avg['rental_count'].max()
        print(f"🔝 피크 시간대: {peak_hour}시 (평균 {peak_value:.1f}회 대여)")
        print()
        
        return hourly_avg
    
    def day_analysis(self):
        """요일별 분석"""
        day_names = ['월', '화', '수', '목', '금', '토', '일']
        
        daily_avg = self.data.groupby('day_of_week')[['rental_count', 'return_count']].mean()
        
        print("📅 요일별 평균 대여/반납량:")
        print("요일  대여   반납")
        print("-" * 20)
        for day in range(7):
            rental = daily_avg.loc[day, 'rental_count']
            return_val = daily_avg.loc[day, 'return_count']
            print(f"{day_names[day]}요일  {rental:5.1f}  {return_val:5.1f}")
        print()
        
        # 평일 vs 주말 비교
        weekday_avg = self.data[self.data['day_of_week'] < 5]['rental_count'].mean()
        weekend_avg = self.data[self.data['day_of_week'] >= 5]['rental_count'].mean()
        
        print(f"📊 평일 vs 주말 비교:")
        print(f"   • 평일 평균: {weekday_avg:.1f}회")
        print(f"   • 주말 평균: {weekend_avg:.1f}회")
        print(f"   • 차이: {abs(weekday_avg - weekend_avg):.1f}회")
        print()
        
        return daily_avg
    
    def station_analysis(self):
        """대여소별 분석"""
        station_stats = self.data.groupby('station_name').agg({
            'rental_count': ['sum', 'mean'],
            'return_count': ['sum', 'mean']
        }).round(2)
        
        # 컬럼명 정리
        station_stats.columns = ['총_대여', '평균_대여', '총_반납', '평균_반납']
        station_stats = station_stats.sort_values('총_대여', ascending=False)
        
        print("🏢 대여소별 이용 현황 (총 대여량 기준):")
        print(station_stats)
        print()
        
        # 인기 대여소 TOP 3
        top3_stations = station_stats.head(3)
        print("🥇 인기 대여소 TOP 3:")
        for i, (station, data) in enumerate(top3_stations.iterrows(), 1):
            print(f"{i}위: {station} (총 {data['총_대여']:.0f}회)")
        print()
        
        return station_stats
    
    def seasonal_analysis(self):
        """계절별 분석"""
        month_names = ['1월', '2월', '3월', '4월', '5월', '6월',
                      '7월', '8월', '9월', '10월', '11월', '12월']
        
        monthly_avg = self.data.groupby('month')[['rental_count', 'return_count']].mean()
        
        print("🌸 월별 평균 대여/반납량:")
        print("월    대여   반납")
        print("-" * 20)
        for month in monthly_avg.index:
            rental = monthly_avg.loc[month, 'rental_count']
            return_val = monthly_avg.loc[month, 'return_count']
            print(f"{month_names[month-1]}  {rental:5.1f}  {return_val:5.1f}")
        print()
        
        # 성수기/비수기 분석
        peak_month = monthly_avg['rental_count'].idxmax()
        low_month = monthly_avg['rental_count'].idxmin()
        
        print(f"📈 성수기: {month_names[peak_month-1]} ({monthly_avg.loc[peak_month, 'rental_count']:.1f}회)")
        print(f"📉 비수기: {month_names[low_month-1]} ({monthly_avg.loc[low_month, 'rental_count']:.1f}회)")
        print()
        
        return monthly_avg
    
    def generate_insights(self):
        """비즈니스 인사이트 생성"""
        print("💡 비즈니스 인사이트 및 제언")
        print("=" * 60)
        
        # 1. 운영 효율성 분석
        total_rental = self.data['rental_count'].sum()
        total_return = self.data['return_count'].sum()
        balance_ratio = total_return / total_rental if total_rental > 0 else 0
        
        print(f"🔄 대여/반납 균형도: {balance_ratio:.2%}")
        if balance_ratio < 0.95:
            print("   ➡️ 제언: 자전거 재배치 시스템 개선 필요")
        elif balance_ratio > 1.05:
            print("   ➡️ 제언: 대여소별 자전거 공급량 점검 필요")
        else:
            print("   ➡️ 평가: 양호한 균형 상태")
        print()
        
        # 2. 시간대별 운영 전략
        hourly_avg = self.data.groupby('hour')['rental_count'].mean()
        peak_hours = hourly_avg[hourly_avg > hourly_avg.mean() + hourly_avg.std()]
        
        print(f"⚡ 피크 시간대: {', '.join([f'{h}시' for h in peak_hours.index])}")
        print("   ➡️ 제언: 피크 시간대 자전거 및 인력 증배 배치")
        print()
        
        # 3. 계절별 운영 전략
        monthly_avg = self.data.groupby('month')['rental_count'].mean()
        peak_months = monthly_avg[monthly_avg > monthly_avg.mean()]
        
        month_names = {1:'1월', 2:'2월', 3:'3월', 4:'4월', 5:'5월', 6:'6월',
                      7:'7월', 8:'8월', 9:'9월', 10:'10월', 11:'11월', 12:'12월'}
        peak_month_names = [month_names[m] for m in peak_months.index]
        
        print(f"🌟 성수기: {', '.join(peak_month_names)}")
        print("   ➡️ 제언: 성수기 대비 자전거 정비 및 확충 계획 수립")
        print()
        
        # 4. 대여소별 전략
        station_stats = self.data.groupby('station_name')['rental_count'].sum().sort_values(ascending=False)
        top_stations = station_stats.head(3).index.tolist()
        
        print(f"🏆 주요 거점: {', '.join(top_stations)}")
        print("   ➡️ 제언: 주요 거점 중심의 마케팅 및 서비스 개선")
        print()
    
    def run_full_analysis(self):
        """전체 분석 실행"""
        # 데이터 생성
        self.generate_sample_data()
        
        # 분석 실행
        self.basic_analysis()
        self.time_analysis()
        self.day_analysis()
        self.station_analysis()
        self.seasonal_analysis()
        self.generate_insights()
        
        print("✅ 분석 완료!")
        print("\n📝 추가 분석 아이디어:")
        print("   • 날씨 데이터와의 상관관계 분석")
        print("   • 지하철역 근처 대여소 이용 패턴 분석")
        print("   • 관광지 근처 대여소 특성 분석")
        print("   • 대여 시간 길이별 이용자 행동 분석")

# 실행 코드
if __name__ == "__main__":
    analyzer = SeoulBikeAnalyzer()
    analyzer.run_full_analysis()

🚲 서울시 따릉이 데이터 분석
📊 데이터 기본 정보:
   • 총 레코드 수: 3,000개
   • 분석 기간: 2024-01-01 ~ 2024-02-11
   • 대여소 수: 10개
   • 총 대여 횟수: 35,352회
   • 총 반납 횟수: 31,983회

📈 기본 통계:
       rental_count  return_count
count       3000.00       3000.00
mean          11.78         10.66
std            9.70          8.90
min            0.00          0.00
25%            3.00          3.00
50%            9.00          8.00
75%           19.00         18.00
max           44.00         40.00

⏰ 시간대별 평균 대여/반납량:
시간  대여   반납
--------------------
 0시    2.7    2.1
 1시    2.4    2.2
 2시    2.4    2.4
 3시    2.5    2.0
 4시    2.4    2.4
 5시    2.6    2.2
 6시    8.5    8.5
 7시   24.9   23.2
 8시   24.8   22.7
 9시   25.4   23.1
10시    8.3    7.4
11시    8.9    7.6
12시   17.2   15.2
13시   16.6   16.1
14시   17.2   15.2
15시    8.6    7.8
16시    8.6    7.4
17시    8.6    8.1
18시   25.9   23.3
19시   25.1   22.9
20시   25.7   22.5
21시    8.8    7.3
22시    2.5    2.4
23시    2.6    2.2

🔝 피크 시간대: 18시 (평균 25.9회 대여)

📅 요일별 평균 대여/반납량:
요일  대여